In [1]:
!pip install gensim

# !pip install --upgrade torch==2.1.1


In [2]:
import pandas as pd
import re
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset

torch.manual_seed(42)



2025-02-28 06:26:28.574818: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-28 06:26:28.574872: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-28 06:26:28.577968: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-28 06:26:28.588955: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-28 06:26:29.639019: W tensorflow/compiler/tf2

In [ ]:
df = pd.read_csv('kaggle_data/YoutubeCommentsDataSet.csv')

# Check the first few rows
df.head()

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive


In [ ]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df = df.drop_duplicates()
df = df.dropna()
df.isnull().sum()

comment      0
sentiment    0
dtype: int64

In [ ]:
def clean_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'http\S+', '', text)  # Remove links
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'#\w+', '', text)  # Remove hashtags
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    return text.strip()

df['cleaned_comment'] = df['comment'].apply(clean_text)
df['word_count'] = df['cleaned_comment'].apply(lambda x: len(str(x).split()))
df['char_count'] = df['cleaned_comment'].apply(lambda x: len(str(x)))

df['sentiment'] = df['sentiment'].replace({'negative': 0, 'neutral': 1, 'positive': 2})

/tmp/ipykernel_1179/1308129695.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'negative': 0, 'neutral': 1, 'positive': 2})


In [ ]:
import random

random.seed(42)

def introduce_misspellings(text, typo_rate=0.5):
    """
    Introduces realistic misspellings into a given text.

    Args:
        text (str): The input text.
        typo_rate (float): The probability of introducing a typo for each word (default: 0.1).

    Returns:
        str: Text with introduced misspellings.
    """
    words = text.split()
    misspelled_words = []

    keyboard_proximity = {  # Example keyboard layout (QWERTY)
        'q': ['w', 'a', '1', '2'],
        'w': ['q', 'e', 'a', 's', '2', '3'],
        'e': ['w', 'r', 's', 'd', '3', '4'],
        'r': ['e', 't', 'd', 'f', '4', '5'],
        't': ['r', 'y', 'f', 'g', '5', '6'],
        'y': ['t', 'u', 'g', 'h', '6', '7'],
        'u': ['y', 'i', 'h', 'j', '7', '8'],
        'i': ['u', 'o', 'j', 'k', '8', '9'],
        'o': ['i', 'p', 'k', 'l', '9', '0'],
        'p': ['o', '[', 'l', ';', '0', '-'],
        'a': ['q', 'w', 's', 'z'],
        's': ['a', 'w', 'e', 'd', 'z', 'x'],
        'd': ['s', 'e', 'r', 'f', 'x', 'c'],
        'f': ['d', 'r', 't', 'g', 'c', 'v'],
        'g': ['f', 't', 'y', 'h', 'v', 'b'],
        'h': ['g', 'y', 'u', 'j', 'b', 'n'],
        'j': ['h', 'u', 'i', 'k', 'n', 'm'],
        'k': ['j', 'i', 'o', 'l', 'm', ','],
        'l': ['k', 'o', 'p', ';', ',', '.'],
        'z': ['a', 's', 'x'],
        'x': ['z', 's', 'd', 'c'],
        'c': ['x', 'd', 'f', 'v'],
        'v': ['c', 'f', 'g', 'b'],
        'b': ['v', 'g', 'h', 'n'],
        'n': ['b', 'h', 'j', 'm'],
        'm': ['n', 'j', 'k', ','],
        '1': ['q', '2'],
        '2': ['1', 'q', 'w', '3'],
        '3': ['2', 'w', 'e', '4'],
        '4': ['3', 'e', 'r', '5'],
        '5': ['4', 'r', 't', '6'],
        '6': ['5', 't', 'y', '7'],
        '7': ['6', 'y', 'u', '8'],
        '8': ['7', 'u', 'i', '9'],
        '9': ['8', 'i', 'o', '0'],
        '0': ['9', 'o', 'p', '-'],
        ',': ['m', 'k', 'l', '.'],
        '.': [',', 'l', ';', '/'],
        '?': ['.', '/'],
        ' ': ['c', 'v', 'b', 'n', 'm'] # considering space as a key too for proximity errors
    }
    alphabet = list('abcdefghijklmnopqrstuvwxyz')

    for word in words:
        if random.random() < typo_rate: # Apply typo to word with probability typo_rate
            if len(word) <= 2: # Avoid misspellings for very short words
                misspelled_words.append(word)
                continue

            operations = ['keyboard_proximity', 'transpose', 'delete', 'duplicate_letter']
            # operation_choice = random.choice(operations)
            
            operation_choice = 'duplicate_letter'

            if operation_choice == 'keyboard_proximity':
                index_to_replace = random.randint(0, len(word) - 1)
                char_to_replace = word[index_to_replace].lower()
                if char_to_replace in keyboard_proximity:
                    possible_replacements = keyboard_proximity[char_to_replace]
                    misspelled_char = random.choice(possible_replacements)
                    if word[index_to_replace].isupper():
                        misspelled_char = misspelled_char.upper()
                    word_list = list(word)
                    word_list[index_to_replace] = misspelled_char
                    misspelled_words.append("".join(word_list))


            elif operation_choice == 'transpose': # Transposition (swap adjacent letters)
                if len(word) >= 2:
                    index_to_swap = random.randint(0, len(word) - 2)
                    word_list = list(word)
                    word_list[index_to_swap], word_list[index_to_swap + 1] = word_list[index_to_swap + 1], word_list[index_to_swap]
                    misspelled_words.append("".join(word_list))
                else:
                    misspelled_words.append(word) # Word too short for transposition


            elif operation_choice == 'insert': # Insertion (add a letter)
                index_to_insert = random.randint(0, len(word))
                char_to_insert = random.choice(alphabet)
                word_list = list(word)
                word_list.insert(index_to_insert, char_to_insert)
                misspelled_words.append("".join(word_list))


            elif operation_choice == 'delete': # Deletion (remove a letter)
                index_to_delete = random.randint(0, len(word) - 1)
                word_list = list(word)
                del word_list[index_to_delete]
                misspelled_words.append("".join(word_list))

            elif operation_choice == 'duplicate_letter': # Duplication (duplicate a letter)
                index_to_duplicate = random.randint(0, len(word) - 1)
                char_to_duplicate = word[index_to_duplicate]
                word_list = list(word)
                word_list.insert(index_to_duplicate, char_to_duplicate) #insert duplicate next to original
                misspelled_words.append("".join(word_list))

            elif operation_choice == 'capitalize': # Capitalization error
                 index_to_capitalize = random.randint(0, len(word) - 1)
                 word_list = list(word)
                 word_list[index_to_capitalize] = word_list[index_to_capitalize].upper() if word_list[index_to_capitalize].islower() else word_list[index_to_capitalize].lower() # Flip case
                 misspelled_words.append("".join(word_list))


        else:
            misspelled_words.append(word) # Keep original word if no typo introduced

    return " ".join(misspelled_words)

# Example usage
# text_data = "This is an example of text data where we will introduce some realistic misspellings."
# misspelled_text = introduce_misspellings(text_data, typo_rate=0.2) # 20% typo rate
# print("Original text:", text_data)
# print("Misspelled text:", misspelled_text)

injected_typos = df.copy()
injected_typos['cleaned_comment'] = df['cleaned_comment'].apply(introduce_misspellings)

# df = pd.concat([df, injected_typos], ignore_index=True)

# df = injected_typos

# df.shape
injected_typos

,comment,sentiment,cleaned_comment,word_count,char_count
0,lets not forget that apple pay in 2014 require...,1,lets noot foorget that apple pay in rrequired ...,56,313
1,here in nz 50 of retailers don’t even have con...,0,here in nz of retailers dont evenn havee connt...,28,159
2,i will forever acknowledge this channel with t...,2,i willl foreveer acknowledge thhis channel wit...,31,183
3,whenever i go to a place that doesn’t take app...,0,whenever i go to a placee thhat doeesnt taake ...,83,441
4,apple pay is so convenient secure and easy to ...,2,applee paay is so convvenient seecure andd eas...,26,135
...,...,...,...,...,...
18403,i really like the point about engineering tool...,2,i rreally likke tthe pooint abouut engineeriin...,52,276
18404,i’ve just started exploring this field and thi...,2,ive jusst starteed exploring this ffield and t...,23,117
18405,excelente video con una pregunta filosófica pr...,1,exceelente video con una preguntaa fiilosfica ...,40,239
18406,hey daniel just discovered your channel a coup...,2,hhey daniiel just discoovered your channel a c...,16,88


In [ ]:
embeddings_model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

In [ ]:
def get_sentence_embedding(sentence, model):
    tokens = word_tokenize(sentence.lower())
    embeddings = []
    for token in tokens:
        if token in model:
            # print(token)
            embeddings.append(model[token])
        # Handling OOV words (options):
        # 1. Skip OOV words (as in the original code):
        else:
            pass  # Or continue
        # 2. Use a zero vector for OOV words:
        # else:
        #     embeddings.append(np.zeros(model.vector_size))
        # 3. Use a special OOV vector (if you have one):
        # else:
        #     embeddings.append(model.wv["<OOV>"])  # If you have an <OOV> token

    if embeddings:
        return np.array(embeddings)  # Average along the 0-axis (words)
    else:
        return np.array([np.zeros(model.vector_size)]) # Or return None if you prefer to filter later.



In [ ]:
def build_dataset(df):
    X_text = df['cleaned_comment']

    tokenizer = Tokenizer(num_words=300000) # Or adjust vocabulary size
    tokenizer.fit_on_texts(X_text)
    X_sequences = tokenizer.texts_to_sequences(X_text)
    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    len(X_sequences[0])

    df['embeddings'] = df['cleaned_comment'].apply(lambda x: get_sentence_embedding(x, embeddings_model))

    print("done")

    embeddings_list = []
    max_len = 0 # Find the maximum sentence length

    for emb in df['embeddings']:
        embeddings_list.append(emb)
        max_len = max(max_len, len(emb))

    print("done")



    chunk_size = 500  # Example chunk size, you might need to experiment with this value

    padded_chunks = []  # List to store padded chunks

    # for i in range(0, len(embeddings_list), chunk_size):
    #     # Extract a chunk of the embeddings_list
    #     chunk = embeddings_list[i:i + chunk_size]

    #     # Pad the current chunk
    #     padded_chunk = pad_sequences(
    #         chunk,
    #         padding="post",
    #         maxlen=max_len,  # Keep maxlen the same as your original command
    #         dtype='float32',
    #         value=0.0
    #     )
    #     padded_chunks.append(padded_chunk) # Add the padded chunk to the list

    #     # print(i)

    # # Concatenate all padded chunks into a single numpy array
    # embeddings_padded = np.concatenate(padded_chunks, axis=0)





    # Pad the sequences so that they all have the same length
    embeddings_padded = pad_sequences(embeddings_list, padding="post", maxlen=max_len, dtype='float32', value = 0.0)
    
    return embeddings_padded

#     print("done")

#     type(embeddings_padded)

#     print("Embeddings Tensor:")
#     print(embeddings_padded[-1])
#     print("Shape of Embeddings Tensor:", embeddings_padded.shape)

#     embeddings_tensor = torch.from_numpy(embeddings_padded)
#     labels_tensor = torch.from_numpy(df['sentiment'].values)

#     torch_dataset = TensorDataset(embeddings_tensor, labels_tensor)
    
#     return torch_dataset

In [ ]:
# embeddings_tensor = torch.from_numpy(build_dataset(df))
# labels_tensor = torch.from_numpy(df['sentiment'].values)
# torch_dataset = TensorDataset(embeddings_tensor, labels_tensor)

embeddings_tensor = torch.from_numpy(build_dataset(injected_typos))
labels_tensor = torch.from_numpy(injected_typos['sentiment'].values)
torch_dataset = TensorDataset(embeddings_tensor, labels_tensor)

save_path = './youtube_comments.pt'

# Save the torch_dataset
# torch.save(torch_dataset, save_path)

# torch_dataset = build_dataset(df)

Found 70694 unique tokens.
done
done


In [ ]:
from torch.utils.data import DataLoader, TensorDataset, random_split

def create_dataloaders(dataset, train_ratio=0.8, batch_size=32):
    """Creates DataLoader objects for training and testing sets.

    Args:
        embeddings: A PyTorch tensor of shape (num_samples, seq_len, embedding_dim).
        labels: A PyTorch tensor of shape (num_samples,).
        train_ratio: The proportion of data to use for training.
        batch_size: The batch size for the DataLoaders.

    Returns:
        A tuple containing the training DataLoader and the testing DataLoader.
    """

    # Split into training and testing sets
    train_size = int(len(dataset) * train_ratio)
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    # Create DataLoaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # Shuffle for training
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) # No need to shuffle for testing.

    return train_dataloader, test_dataloader


train_dataloader, test_dataloader = create_dataloaders(torch_dataset, train_ratio=0.8, batch_size=32)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SentimentCNN(nn.Module):
    def __init__(self, embedding_dim, num_filters, kernel_sizes, num_classes):
        super(SentimentCNN, self).__init__()
        # Convolutional layers
        self.conv_layers = nn.ModuleList([
            nn.Conv1d(in_channels=embedding_dim,
                      out_channels=num_filters,
                      kernel_size=ks)
            for ks in kernel_sizes
        ])
        # Fully connected layer
        self.fc = nn.Linear(len(kernel_sizes) * num_filters, num_classes)
        # Dropout for regularization
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # x shape: (batch_size, seq_len, embedding_dim)
        # CNN expects input shape (batch_size, embedding_dim, seq_len)
        x = x.transpose(1, 2)

        # Apply convolutional layers and pooling
        pooled_outputs = []
        for conv in self.conv_layers:
            conved = F.relu(conv(x)) # Convolution and ReLU activation
            pooled = F.max_pool1d(conved, conved.shape[2]).squeeze(2) # Max pooling over sequence length
            pooled_outputs.append(pooled)

        # Concatenate pooled outputs
        concatenated = torch.cat(pooled_outputs, dim=1)
        # Apply dropout
        dropped = self.dropout(concatenated)
        # Fully connected layer
        output = self.fc(dropped)
        return output

In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_auc_score, #For binary classification only
)
# ... (Model loading and setup as before - load model, set to eval mode) ...

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = torch.load('epoch_typo_15.pth', map_location=device, weights_only=False) #map_location is important!
model.to(device) #Move the model to the correct device.

model.eval()  # VERY IMPORTANT: Sets the model to evaluation mode.  Crucial for correct inference.


def evaluate_model(model, dataloader, device):
    """Evaluates the model on the given dataloader and returns recall and precision."""

    model.eval()  # Set to evaluation mode
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch_embeddings, batch_labels in dataloader:
            batch_embeddings = batch_embeddings.to(device)
            batch_labels = batch_labels.to(device)

            outputs = model(batch_embeddings)
            probabilities = F.softmax(outputs, dim=1)  # Get probabilities
            predictions = torch.argmax(probabilities, dim=1)  # Get predicted labels

            all_predictions.extend(predictions.cpu().numpy())  # Move to CPU for sklearn
            all_labels.extend(batch_labels.cpu().numpy())
            
            # print('done batch')

    # Calculate recall and precision using sklearn    
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='macro', zero_division=1) #Handles zero division
    recall = recall_score(all_labels, all_predictions, average='macro', zero_division=1) #Handles zero division
    f1 = f1_score(all_labels, all_predictions, average='macro', zero_division=1) #Handles zero division
    conf_matrix = confusion_matrix(all_labels, all_predictions)
    class_report = classification_report(all_labels, all_predictions, zero_division=1) #Handles zero division

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")
    # print("Confusion Matrix:\n", conf_matrix)
    print("Classification Report:\n", class_report)

    return recall, precision



# Example usage:
recall, precision = evaluate_model(model, test_dataloader, device)  # Replace test_dataloader with your test data's dataloader

# print(f"Recall: {recall}")
# print(f"Precision: {precision}")

Accuracy: 0.688951048951049
Precision: 0.6489408252611937
Recall: 0.6319116633122085
F1-score: 0.6209166817452871
Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.41      0.48       458
           1       0.49      0.78      0.60       922
           2       0.87      0.71      0.78      2195

    accuracy                           0.69      3575
   macro avg       0.65      0.63      0.62      3575
weighted avg       0.74      0.69      0.70      3575

